In [2]:
using Distributions
mu_1 = 8.0
sig_1 = sqrt(2.0)
mu_2 = 12.0
sig_2 = sqrt(2.0)
beta = sqrt(3.0)

f_1(s1::Int) = pdf(Normal(mu_1, sig_1), s1)
f_2(s1::Int, p1::Int) = pdf(Normal(s1, beta), p1)
f_3(s2::Int) = pdf(Normal(mu_2, sig_2), s2)
f_4(s2::Int, p2::Int) = pdf(Normal(s2, beta), p2)
f_5(p1::Int, p2::Int, d::Int) = (p1 - p2 == d) ? 1.0 : 0.0 
f_6(d::Int) = if d > 0 ? 1.0 : 0.0 end

f_6 (generic function with 1 method)

In [3]:
# marginalization for s1

function marginalize_s1(N::Int, s1::Int)
    sum = 0.0
    for s2 in 1:N
        inner_sum_s2 = 0.0
        for p1 in 1:N
            inner_sum_p1 = 0.0
            for p2 in 1:p1 # its not wrong
                inner_sum_p1 += f_4(s2, p2)
            end
            inner_sum_p1 *= f_2(s1, p1)
            inner_sum_s2 += inner_sum_p1
        end
        inner_sum_s2 *= f_3(s2)
        sum += inner_sum_s2
    end
    return sum * f_1(s1)
end

marginalize_s1 (generic function with 1 method)

In [4]:
marginalize_s1(20, 10)

0.030819077827076418

In [5]:
marginalize_s1(20, 8)

0.03011650250443023

In [6]:
for i in 1:20
    n = marginalize_s1(20, i) * 1000
    s = (i < 10) ? " " : "" 
    s = s * string(i)
    s = s * repeat(".", Int(floor(n)))
    println(s)
end

 1
 2
 3
 4
 5
 6..
 7............
 8..............................
 9.........................................
10..............................
11............
12..
13
14
15
16
17
18
19
20


In [7]:
# marginalization for s1

function marginalize_s2(N::Int, s2::Int)
    sum = 0.0
    for s1 in 1:N
        inner_sum_s1 = 0.0
        for p1 in 1:N
            inner_sum_p1 = 0.0
            for p2 in 1:p1 # its not wrong
                inner_sum_p1 += f_4(s2, p2)
            end
            inner_sum_p1 *= f_2(s1, p1)
            inner_sum_s1 += inner_sum_p1
        end
        inner_sum_s1 *= f_1(s2)
        sum += inner_sum_s1
    end
    return sum * f_3(s2)
end

marginalize_s2 (generic function with 1 method)

In [8]:
for i in 1:20
    n = marginalize_s2(20, i) * 500
    s = (i < 10) ? " " : "" 
    s = s * string(i)
    s = s * repeat(".", Int(floor(n)))
    println(s)
end

 1
 2
 3
 4
 5
 6
 7
 8........
 9....................................
10.......................................................
11..............................
12......
13
14
15
16
17
18
19
20


In [9]:
function marginalize_p1(N::Int, p1::Int)
    sum = 0.0
    for s1 in 1:N
        inner_sum_s2= 0.0
        for s2 in 1:N
            inner_sum_p2 = 0.0
            for p2 in 1:p1
                inner_sum_s2 += f_4(s2, p2)
            end
            inner_sum_s2 += (f_3(s2) * inner_sum_s2)
        end
        sum += (f_1(s1) * f_2(s1,p1) * inner_sum_s2) 
    end
    return sum
end

marginalize_p1 (generic function with 1 method)

In [10]:
for i in 1:20
    n = marginalize_p1(20, i) * 10
    s = (i < 10) ? " " : "" 
    s = s * string(i)
    s = s * repeat(".", Int(floor(n)))
    println(s)
end

 1
 2
 3
 4..
 5.......
 6...............
 7.........................
 8................................
 9.................................
10...........................
11..................
12.........
13....
14.
15
16
17
18
19
20


In [11]:
function marginalize_p2(N::Int, p2::Int)
    sum = 0.0
    for s1 in 1:N
        inner_sum_s2 = 0.0
        for s2 in 1:N
            inner_sum_p1 = 0.0
            for p1 in p2:N
                inner_sum_p1 += f_2(s1, p1)
            end
            inner_sum_s2 += (f_3(s2) * f_4(s2, p2))
        end
        sum += (inner_sum_s2 * f_1(s1))
    end
    return sum
end

marginalize_p2 (generic function with 1 method)

In [12]:
x = 1:4

1:4

In [13]:
function marginalize_d(N::Int, d::Int)
    if d <= 0
        return 0.0
    end
    sum = 0.0
    for s1 in 1:N
        inner_sum_s2 = 0.0
        for s2 in 1:N
            inner_sum_p1 =0.0
            for p1 in d:N
                inner_sum_p1 += f_2(s1, p1) * f_4(s2, p1 - d)
            end
            inner_sum_s2 += (inner_sum_p1 * f_3(s2))
        end
        sum = f_1(s1) * inner_sum_s2
    end
    return sum
end

marginalize_d (generic function with 1 method)

In [14]:
for i in 1:20
    n = marginalize_d(20, i) * (10.0^(19)) 
    s = (i < 10) ? " " : "" 
    s = s * string(i)
    s = s * repeat(".", Int(floor(n)))
    println(s)
end

 1....
 2.........
 3..................
 4...............................
 5..............................................
 6...........................................................
 7................................................................
 8...........................................................
 9..............................................
10..............................
11................
12.......
13..
14
15
16
17
18
19
20


In [15]:
#forward
function_mf1_s1(s1::Int) = f_1(s1)
function_mf3_s2(s2::Int) = f_3(s2)
function_f_p_s1(s1::Int) = function_mf1_s1(s1) 
function_f_p_s2(s2::Int) = function_mf3_s2(s2)
function_mf2_p1(N::Int, p1::Int) = sum([f_2(s1, p1) * function_f_p_s1(s1) for s1 in 1:N])
function_mf4_p2(N::Int, p2::Int) = sum([f_4(s2, p2) * function_f_p_s2(s2) for s2 in 1:N])
function_f_p_p1(N::Int, p1::Int) = function_mf2_p1(N, p1) 
function_f_p_p2(N::Int, p2::Int) = function_mf4_p2(N, p2)
function_mp1_f5(N::Int, p1::Int) = function_f_p_p1(N, p1)
function_mp2_f5(N::Int, p2::Int) = function_f_p_p2(N, p2)
function_mf5_d(N::Int, d::Int) = sum((f_5(p1, p1 - d, d) *  function_mp1_f5(N,p1) * function_mp2_f5(N,p1 - d)) for p1 in d:N)
function_f_p_d(N::Int, d::Int) = function_mf5_d(N,d)

#backward
#observer d = 1
function_mf6_d(d::Int) = (d > 0.0) ? 1.0 : 0.0
function_p_d(N::Int, d::Int) = function_mf6_d(d) > 0.0 ? function_mf5_d(N,d) : 0.0
function_md_f5(N::Int, d::Int) = (function_mf5_d(N, d) > 0.0 )  ? (function_p_d(N, d) / function_mf5_d(N, d)) : 0.0
function_mf5_p1(N::Int, p1::Int) = sum((function_md_f5(N, d) *  function_mp2_f5(N, p1 - d)) for d in 1:N) # d = p1 - p2, p2 = p1 - d
function_mf5_p2(N::Int, p2::Int) = sum((function_md_f5(N, d) *  function_mp1_f5(N, p2 + d)) for d in 1:N) # p1 = p2 + d
function_p_p1(N::Int, p1::Int ) = function_mf2_p1(N, p1) * function_mf5_p1(N, p1) # sum()
function_p_p2(N::Int, p2::Int ) = function_mf4_p2(N, p2) * function_mf5_p2(N, p2)
function_mf2_s1(N::Int, s1::Int) = sum( ( f_2(s1, p1) * (function_p_p1(N,p1) / function_mf2_p1(N, p1)) ) for p1 in 1:N)
function_mf4_s2(N::Int, s2::Int) = sum( ( f_4(s2, p2) * (function_p_p2(N,p2) / function_mf4_p2(N, p2)) ) for p2 in 1:N)
function_p_s1(N::Int, s1::Int) = function_mf1_s1(s1) * function_mf2_s1(N,s1)
function_p_s2(N::Int, s2::Int) = function_mf3_s2(s2) * function_mf4_s2(N, s2)


function_p_s2 (generic function with 1 method)

In [16]:
function posterior_s1(N::Int, s1::Int)
    cach = zeros(Float64, 12, N)
    mf1_s1(s1::Int) = f_1(s1)
mf3_s2(s2::Int) = f_3(s2)
f_p_s1(s1::Int) = mf1_s1(s1) 
f_p_s2(s2::Int) = mf3_s2(s2)
mf2_p1(N::Int, p1::Int) = sum([f_2(s1, p1) * f_p_s1(s1) for s1 in 1:N])
mf4_p2(N::Int, p2::Int) = sum([f_4(s2, p2) * f_p_s2(s2) for s2 in 1:N])
f_p_p1(N::Int, p1::Int) = mf2_p1(N, p1) 
f_p_p2(N::Int, p2::Int) = mf4_p2(N, p2)
mp1_f5(N::Int, p1::Int) = f_p_p1(N, p1)
mp2_f5(N::Int, p2::Int) = f_p_p2(N, p2)
mf5_d(N::Int, d::Int) = sum((f_5(p1, p1 - d, d) *  mp1_f5(N,p1) * mp2_f5(N,p1 - d)) for p1 in d:N)
f_p_d(N::Int, d::Int) = mf5_d(N,d)


    return cach
end

posterior_s1(20, 10)

12×20 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [18]:
for i in 1:20
    #n = ((mf5_d(20, i) > 0.0 )  ? (p_d(20, i) / mf5_d(20, i)) : 0.0)  * (10.0^(1)) 
    #s = p_s1(20, i)(i < 10) ? " " : "" 
    s = s * string(i)
    s = s * repeat(".", Int(floor(n)))
    println(s)
end

UndefVarError: UndefVarError: `s` not defined in local scope
Suggestion: check for an assignment to a local variable that shadows a global of the same name.

In [19]:
for i in 1:20
    n = p_d(20, i)
    n *= 10.0^19
    s = string(i)
    s = repeat(" ", Int(floor(3-length(s)))) * s
    s = s * repeat(".", max(Int(floor(n)), 0))
    println(s)
end

UndefVarError: UndefVarError: `p_d` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [20]:
for i in -5:20
    n = marginalize_d(20, i)
    n *= 10.0^19
    s = string(i)
    s = repeat(" ", Int(floor(3-length(s)))) * s
    s = s * repeat(".", max(Int(floor(n)), 0))
    println(s)
end

 -5
 -4
 -3
 -2
 -1
  0
  1....
  2.........
  3..................
  4...............................
  5..............................................
  6...........................................................
  7................................................................
  8...........................................................
  9..............................................
 10..............................
 11................
 12.......
 13..
 14
 15
 16
 17
 18
 19
 20


In [21]:
10^19

-8446744073709551616

In [22]:
# calculating using vector
#forward

function calculateAllMarginalsViaMsg()
    N = 20
    mf1_s1 = [f_1(s1) for s1 in 1:N]
    mf3_s2 = [f_3(s2) for s2 in 1:N]
    f_p_s1 = copy(mf1_s1)
    f_p_s2 = copy(mf3_s2)
    mf2_p1 = [sum([f_2(s1, p1) * f_p_s1[s1] for s1 in 1:N]) for p1 in 1:N]
    mf4_p2 = [sum([f_4(s2, p2) * f_p_s2[s2] for s2 in 1:N]) for p2 in 1:N]
    f_p_p1 = copy( mf2_p1 )
    f_p_p2 = copy(mf4_p2)
    mp1_f5 = copy(f_p_p1)
    mp2_f5 = copy(f_p_p2)
    mf5_d = [sum([f_5(p1, p1 - d, d) *  mp1_f5[p1] * mp2_f5[p1 - d] for p1 in max(d + 1,1): min(N,N + d) ];init = 0.0) for d in -N:N]
    f_p_d = copy(mf5_d)

    #backward
    #observer d = 1
    mf6_d = [(d > 0.0) ? 1.0 : 0.0 for d in -N:N]
    p_d =  [ mf6_d[d + N + 1] > 0.0 ? mf5_d[d+ N + 1] : 0.0 for d in -N:N]
    md_f5 = [(mf5_d[d + N + 1] > 0.0 ) ? (p_d[d + N + 1] / mf5_d[d + N + 1]) : 0.0 for d in -N:N]
    mf5_p1 = [sum([md_f5[d+N+1] * mp2_f5[p1 - d] for d in 1:(p1-1)]; init=0.0) for p1 in 1:N]
    mf5_p2 = [ sum([md_f5[d + N + 1 ] *  mp1_f5[p2 + d] for d in 1:(N - p2)]; init = 0.0) for p2 in 1:N ] # p2 + d = p1 / p1 in 1:N -> d 1:N-p2 
    p_p1 = [ (mf2_p1[p1] * mf5_p1[p1]) for p1 in 1:N]
    p_p2 = [ (mf4_p2[p2] * mf5_p2[p2]) for p2 in 1:N]
    mf2_s1 = [ sum([ f_2(s1, p1) * (p_p1[p1] / mf2_p1[p1]) for p1 in 1:N ]) for s1 in 1:N ]
    mf4_s2 = [sum([  f_4(s2, p2) * (p_p2[p2] / mf4_p2[p2]) for p2 in 1:N]) for s2 in 1:N]
    p_s1 = [mf1_s1[s1] * mf2_s1[s1] for s1 in 1:N]
    p_s2 = [mf3_s2[s2] * mf4_s2[s2] for s2 in 1:N]
end


calculateAllMarginalsViaMsg (generic function with 1 method)

In [23]:
function_p_s2(20, 10)

0.019419424014012632

In [24]:
p_s2[10]

UndefVarError: UndefVarError: `p_s2` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [25]:
for i in [[ (d, p1, p1 - d) for p1 in max(d + 1,1): min(N,N + d) ] for d in -N:N] 
    if length(i) > 0
        if i[1][1] >= 0
            println(i)
        end
    end
end

UndefVarError: UndefVarError: `N` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [26]:
[sum([md_f5[d + N + 1] * mp2_f5[p1 - d] for d in 1:(p1-1)]; init=0.0) for p1 in 1:N]

UndefVarError: UndefVarError: `N` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [27]:
[ sum([md_f5[d + N + 1 ] *  mp1_f5[p2 + d] for d in 1:(N - p2)]; init = 0.0) for p2 in 1:N ]

UndefVarError: UndefVarError: `N` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [28]:
[ [ (d, p2+d,p2) for d in 1:(N - p2)] for p2 in 1:N ]

UndefVarError: UndefVarError: `N` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [29]:
function calculateAllMarginals()
    N = 20
    for i in 1:N marginalize_s1(20, i) end
    for i in 1:N marginalize_s2(20, i) end
    for i in 1:N marginalize_p1(20, i) end
    for i in 1:N marginalize_p2(20, i) end
    for i in -N:N marginalize_d(20, i) end
end

calculateAllMarginals (generic function with 1 method)

In [30]:
calculateAllMarginalsViaMsg()

20-element Vector{Float64}:
 1.2433702695580425e-14
 3.0749122243278523e-12
 3.9556071702102667e-10
 2.771480319072429e-8
 1.0915772369383887e-6
 2.445226609834744e-5
 0.0003106519809949374
 0.0022189656345265344
 0.008829806807992736
 0.019419423106674247
 0.023454777491353683
 0.015479072673902774
 0.005559781360420571
 0.001083478045063023
 0.00011427960953829765
 6.511184694361758e-6
 2.0008636205276457e-7
 3.31196037569514e-9
 2.949619834706995e-11
 1.4115300110420243e-13

In [31]:
calculateAllMarginals()

In [33]:
t1 = @belapsed calculateAllMarginalsViaMsg()
t2 = @belapsed calculateAllMarginals()

println("ViaMsg: $t1 seconds")
println("Direct: $t2 seconds")
println("Speedup: ", t2 / t1, "× (ViaMsg vs Direct)")

ViaMsg: 0.000309416 seconds
Direct: 0.096640208 seconds
Speedup: 312.3309977506011× (ViaMsg vs Direct)
